In [2]:
import pandas as pd
from gurobipy import Model, GRB
import numpy as np

In [3]:
# Read the cost matrix and demand distributions
cost_matrix = pd.read_csv('/Users/mahinbindra/Downloads/cost_matrix.csv')
distributions = pd.read_csv('/Users/mahinbindra/Downloads/distributions.csv')

In [4]:
# Number of scenarios to generate
num_scenarios = 1000

In [5]:
# Initialize a dictionary to store the demand scenarios
demand_scenarios = {}

In [6]:
# Generate demand scenarios using Monte Carlo simulation for all retailers
for index, row in distributions.iterrows():
    retailer = index + 1  # Assuming retailers are 1-indexed
    n, p = int(row['n']), row['p']
    
    # Generate random demand values from a binomial distribution
    demand_scenarios[retailer] = np.random.binomial(n, p, num_scenarios)

In [7]:
# Check the output for all retailers by showing the first 5 demand scenarios for each
demand_scenario = {retailer: scenarios[:5] for retailer, scenarios in demand_scenarios.items()}

In [8]:
trans_cost = {(i, j): cost_matrix.iat[i, j] for i in range(len(cost_matrix)) 
              for j in range(len(cost_matrix.columns)) if i != j}

In [9]:
def setup_gurobi_model(demand_scenarios, trans_cost):
    # Initialize the Gurobi model
    model = gp.Model('BaffinBayInventoryOptimization')

    # Parameters
    C = 24.44  # unit ordering cost
    P = 49.99  # unit selling price
    retailers = range(1, len(demand_scenarios) + 1)  # Retailers numbered from 1 to n
    num_scenarios = len(next(iter(demand_scenarios.values())))  # Number of scenarios

    # First-stage decision variables: Order quantities for each retailer
    y = model.addVars(retailers, name="y", lb=0)

    # Second-stage decision variables: Transshipment quantities from retailer j to retailer i under scenario k
    x = model.addVars(retailers, retailers, range(num_scenarios), name="x", lb=0)

    # Objective function: Minimize the expected total cost
    model.setObjective(
        C * gp.quicksum(y[i] for i in retailers) +
        gp.quicksum(gp.quicksum(trans_cost.get((i-1, j-1), 0) * x[i, j, k]
                                for i in retailers for j in retailers if i != j)
                    for k in range(num_scenarios)) / num_scenarios,
        GRB.MINIMIZE
    )

In [19]:
def setup_gurobi_model(demand_scenarios, trans_cost):
    # Initialize the Gurobi model
    model = gp.Model('BaffinBayInventoryOptimization')

    # Parameters
    C = 24.44  # unit ordering cost
    P = 49.99  # unit selling price
    retailers = range(1, len(demand_scenarios) + 1)  # Retailers numbered from 1 to n
    num_scenarios = len(next(iter(demand_scenarios.values())))  # Number of scenarios

    # First-stage decision variables: Order quantities for each retailer
    y = model.addVars(retailers, name="y", lb=0)

    # Second-stage decision variables: Transshipment quantities from retailer j to retailer i under scenario k
    x = model.addVars(retailers, retailers, range(num_scenarios), name="x", lb=0)

    # Objective function: Minimize the expected total cost
    model.setObjective(
        C * gp.quicksum(y[i] for i in retailers) +
        gp.quicksum(gp.quicksum(trans_cost.get((i-1, j-1), 0) * x[i, j, k]
                                for i in retailers for j in retailers if i != j)
                    for k in range(num_scenarios)) / num_scenarios,
        GRB.MINIMIZE
    )

    # Constraints
    # Demand fulfillment constraints
    for k in range(num_scenarios):
        for i in retailers:
            model.addConstr(
                y[i] + gp.quicksum(x[j, i, k] for j in retailers if j != i) -
                gp.quicksum(x[i, j, k] for j in retailers if j != i) >= demand_scenarios[i][k],
                name=f"demand_fulfillment_{i}_{k}"
            )

    # Transshipment constraints
    for k in range(num_scenarios):
        for i in retailers:
            for j in retailers:
                if i != j:
                    model.addConstr(
                        x[i, j, k] <= y[j] - demand_scenarios[j][k],
                        name=f"transshipment_{j}_{i}_{k}"
                    )

    # Solve the model
    model.optimize()

    # Output the solution
    if model.status == GRB.OPTIMAL:
        print('The optimal solution:')
        for i in retailers:
            print(f'Order quantity for retailer {i}: {y[i].X}')
        for k in range(num_scenarios):
            for i in retailers:
                for j in retailers:
                    if i != j and x[i, j, k].X > 0:
                        print(f'Scenario {k}: Transship from retailer {j} to retailer {i}: {x[i, j, k].X}')
    else:
        print('No optimal solution found')

# Call the function with the actual demand scenarios and trans_cost dictionary
setup_gurobi_model(demand_scenario, trans_cost)

NameError: name 'gp' is not defined